# RNA-Seq Analysis Training Demo

## Overview

This tutorial will show you the following workflow that is used to generate the list of differentially expressed genes. Running the code in this tutorial should take approximately 15 minutes.

This overflow centers roughly around the useage of the deseq2 package. Documentation for this package can be found here: https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

![RNA-Seq workflow](images/count-workflow.png)

## Learning Objectives

* **Data Ingestion and Preprocessing:** Reading and preprocessing RNA-Seq gene count data, including handling zero read counts and creating a suitable data matrix for DESeq2.  This involves downloading data from a cloud storage bucket.

* **Experimental Design Specification:** Defining the experimental design (e.g., treatment groups) using a data frame to guide the differential expression analysis.

* **DESeq2 Object Creation and Analysis:** Creating a DESeq2 object from the count data and experimental design, performing pre-filtering, and running the DESeq2 pipeline for differential expression analysis.

* **Data Normalization:** Applying rlog transformation to normalize the RNA-Seq count data to reduce technical variation between samples.  Visualizing the effects of normalization using histograms, boxplots, and pairwise comparison plots.

* **Principal Component Analysis (PCA):** Performing PCA to visualize sample relationships and identify potential batch effects or outliers.

* **Contrast Comparisons:** Performing pairwise comparisons between treatment groups (e.g., wildtype vs. bacteriophage infected) using DESeq2 to identify differentially expressed genes.  Interpreting the results, including p-values and adjusted p-values.

* **Result Annotation and Export:** Merging differential expression results with gene annotation data to add gene identifiers and descriptions. Exporting the annotated results to a file.

* **Visualization of Results:** Creating volcano plots, MA plots, and heatmaps to visualize differential gene expression results and explore gene expression patterns within groups of interest.

* **Understanding RNA-Seq Workflow:**  The notebook positions itself within a broader RNA-Seq workflow, providing links to other notebooks that cover earlier steps (e.g., sequence read trimming, mapping, quantification).

## Prerequisites

**Software and Libraries:**

* The core analysis is done using R.  
* The notebook uses several R packages that need to be installed: `BiocManager`, `ComplexHeatmap`, `DESeq2`, `EnhancedVolcano`, `dplyr`, `pheatmap`, `ggrepel`, and `ggfortify`.  `BiocManager` is used to install Bioconductor packages.

**APIs**

* **`gsutil` (or `curl`):**  To download files from Google Cloud Storage. `gsutil` is generally preferred for efficient GCS interaction.
* **`curl`:** This is used as a secondary method for file download, but `gsutil` is the primary method used in the notebook.



## Get Started

### STEP 1: Library Installation

First install the R packages that will be used. <strong>This installation may take around 15 minutes.</strong>

In [ ]:
###install the libraries first, if required
install.packages("BiocManager")

if (!require("ComplexHeatmap"))
    BiocManager::install("ComplexHeatmap")

if (!require("DESeq2"))
    BiocManager::install("DESeq2")

if (!require("EnhancedVolcano"))
    BiocManager::install('EnhancedVolcano')

if (!require("dplyr"))
    install.packages("dplyr")

if (!require("pheatmap"))
    install.packages("pheatmap")

if (!require("ggrepel"))
    install.packages("ggrepel")

if (!require("ggfortify"))
    install.packages("ggfortify")


###load the libraries
library(DESeq2)
library(dplyr)
library(ComplexHeatmap)
library(EnhancedVolcano)
library(ggplot2)
library(ggrepel)
library(ggfortify)

### STEP 2: Reading in Data

Next read-in the gene count data that will be analyzed.

This read-in step often involves reformatting or adjusting the data.

To make things easier, we created a matrix, or a table, of just our read-count data -- excluding the non-numerical metadata columns. This makes it easy for us to feed this table into the deseq2 tool later on.

Additionally, later on we will use normalization involving logarithms. So, to prevent errors, we will replace all readcount values of '0' in our data, with '1'. This will change the data only slightly, and will prevent these zeroes from causing an 'undefined' or 'N/A' math error in the logarithm normalization step.


In [ ]:
system("gsutil cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/salmon/SRP300216_merged_quants.txt --output data/gene_counts/SRP300216_genecounts.txt")

In [ ]:
#download genecounts and annotation file from cloud storage bucket
system("mkdir -p data/gene_counts", intern=TRUE)
system("mkdir -p data/reference", intern=TRUE)
system("gsutil cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/salmon/SRP300216_merged_quants.txt data/gene_counts/SRP300216_genecounts.txt")
#try downloading with curl
system("curl https://storage.googleapis.com/nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/reference/GCF_001632805.1_ASM163280v1_feature_table.txt --output data/reference/GCF_001632805.1_ASM163280v1_feature_table.txt", intern=TRUE)


In [ ]:
#read-in the raw gene count file to a dataframe variable we named 'read_counts'
read_counts <- read.table('data/gene_counts/SRP300216_genecounts.txt',head=TRUE)

#replace all count entries of '0' in the raw gene count file with '1'.
read_counts[read_counts==0] <- 1

#assign the numerical read counts to a matrix variable we named 'rnaseqMatrix'
rnaseqMatrix <- round(read_counts[,c(2:13)])

#label the rownames of this matrix with the rowname column from the gene count dataframe.
rownames(rnaseqMatrix) <- read_counts[,1]

head(rnaseqMatrix)


### STEP 4: Specifying Experimental Design

Next specify the experimental design. 

The deseq2 tool will later use this design to group samples together, and output information about the statistical differences in gene expression between these specified groups.

In [ ]:
# define the sample experimental design, in this case 6 wildtype and 6 bacteriophage infected samples
samples_treatment <- data.frame(matrix(c(rep("WT",6),rep("BPs_lysogen",6)),ncol=1))
samples_ID <- data.frame(matrix(colnames(rnaseqMatrix),ncol=1))
samples <- cbind(samples_ID,samples_treatment)
names(samples) <- c("ID","Treatment")
rownames(samples) <- samples[,1]

print("An example of how a deseq2 experimental design table might look.")
samples



### STEP 5: Creating Deseq2 Object

Now use the treatment design matrix in combination with the readcount matrix to create a deseq2 object. 

Once created, this is also a good opportunity to filter out lowly expresseed genes, and to inspect the pre-normalized data using pairwise comparison plots.

Finally, the deseq2 analysis can be run on the deseq2 object.

In [ ]:
#Create the DEseq2DataSet object
deseq2Data <- DESeqDataSetFromMatrix(countData = rnaseqMatrix, colData = samples, design = ~ Treatment)

# Perform pre-filtering of the data
deseq2Data <- deseq2Data[rowSums(counts(deseq2Data)) > 10, ]

# Inspect pre-normalized data 
pairs(log2((counts(deseq2Data)+1)))
boxplot(log2((counts(deseq2Data)+1)))

# Run pipeline for differential expression steps
deseq2Data <- DESeq(deseq2Data)


### STEP 6: Normalize Data

Now normalize the data using the 'regular normalization' function from deseq2.

Normalization of gene readcounts across samples will help to minimize sample variation noise and ensure more accurate comparisons.

In [ ]:
#perform rlog normalization on deseq2 object
rld <- rlog(deseq2Data, blind=FALSE)

#a handy variable so we can easily reference the rlog genecount matrix in the future, as deseq objects contain many tables of data.
rlogcounts <- assay(rld)
rownames(rlogcounts) <- rownames(deseq2Data)

head(rlogcounts)

Normalization of data can is often verified by inspecting by various plots, for instance histograms:

In [ ]:
hist(rlogcounts)

Boxplots:

In [ ]:
boxplot(rlogcounts)

Or pairwise comparison plots.

In [ ]:
pairs(rlogcounts)

Compare the plots of the pre-normalized and normalized data to see the effect of normalization.

Notice how the samples in the box plot, and in-group samples in the pair plots display similar distributions.

Looking at normalized plots can be an easy way to look out for any obvious inconsistencies or errors in the sample data.

### STEP 7: PCA Plot

We can now begin to produce plots to further analyze the differences between groups.

PCA plots can be an effective way to visualize variation within and between sample sets.

In [ ]:
pca_counts <- prcomp(t(rlogcounts))
autoplot(pca_counts, data = samples, colour="Treatment")

Plots can be output as above, or also saved as pdf or image files, as below. Try to navigate and click to the created file, after you run the code.

In [ ]:
pdf("data/gene_counts/PCA_Plot.pdf")
autoplot(pca_counts, data = samples, colour="Treatment")
dev.off()

### STEP 8: Contrast Comparisons

Contrast comparisons between two treatment groups can be performed using the deseq2 tool to identify differentially up and down regulated genes.

Using R, these results can be summarized, reordered, and/or trimmed.

In [ ]:
#Output the results of comparing WT and BPs_lysogen group
res_WT_BPs <- results(deseq2Data, contrast=c("Treatment","WT","BPs_lysogen"))
summary(res_WT_BPs)
print("Number of genes under a p value of 0.05")
sum(res_WT_BPs$pvalue < 0.05, na.rm=TRUE)
print("Number of genes under a p adjusted value of 0.05")
sum(res_WT_BPs$padj < 0.05, na.rm=TRUE)


#deseq2 optionally output some padj values as 'NA' when genecounts meet certain criteria
#that criteria can be found here: https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA
#for practical uses, one could replace the NA values with '1', essentially NA values not significant.
res_WT_BPs$padj[is.na(res_WT_BPs$padj)] <- 1

### STEP 9: Annotating and Exporting Results

These differentially expressed genes can be annotated and exported.

Combining gene counts with gene annotation isn't always simple, and will vary heavily depending on the annotation file itself.

For example, when merging genecounts by 'gene id' with feature tables from NCBI's assembly database, its very common to have duplicate rows, as a single 'gene id' may match to multiple features. 

Depending on how you want to use your annotated results, you may decide to further reformat your table in whatever suits your further analysis best, for instance by downloading and using a spreadsheet editor.

Because the transcript table for m chelonae we have here is so simple, and contains essentially only one 'CDS' or '..RNA' feature for every gene identifier we are using, it is easy for us to reformat it here in the way we want just using R code. However, it is important to note that, again, this is very often not the case, and reformatting will vary depending on both the annotation file itself, and what that annotation file will be used for.

In [ ]:
#read-in the annotation file (see extended tutorial for more info on how to get annotation files) and store it into a variable.
annotation <- read.table("data/reference/GCF_001632805.1_ASM163280v1_feature_table.txt", header=TRUE,sep='\t', quote="\"",comment.char = "")

#put the previous deseq results into an easier to manipulate data.frame format
results <- data.frame(res_WT_BPs)

#merge the annotation with the deseq2 contrast comparison table
results.annot <- merge(results,annotation,by.x=0,by.y=17,all.x=TRUE)



#Open the annotation file above to see the format. If you further download and examine in a spreadsheet editor,
#you can see there is one CDS or ..RNA identifier for each 'gene' feature
#this makes reformatting in R simple by just removing every 'gene' feature, and leaving the more detailed 'cds' or '...RNA' feature.
#this is very often not always the case, but simple and easy here.
results.annot <- results.annot[annotation$X..feature!='gene',]

#labeling the columns properly after the merge
colnames(results.annot)[1] <- 'locus_tag'

#write out the file
write.table(results.annot,"data/gene_counts/WT_BPs_DESeq2_annot.txt",row.names=FALSE,sep="\t")

head(results.annot)


### STEP 10: More Plots and Clustering Data

Finally, although several other different kind of analysis can be done, it is common to plot the results of differential gene expression analysis in forms such as volcano, MA, and heatmaps. These plots help one to further examine the output list of differentially expressed genes in different ways, and potentially identify or further investigate candidate gene(s).

In [ ]:
#volcano and ma plots can be used to visualize significant differences in gene expression
results_volc_df <- results.annot
EnhancedVolcano(results_volc_df,
    lab = results_volc_df[,1],
    x = 'log2FoldChange',
    y = 'padj')

#ma plot
plotMA(res_WT_BPs)


#heatmaps can used to look at clustering and expression of various gene lists.
filtered_res <- results.annot %>% filter(padj < 0.05)
filtered_res_counts <- subset(rlogcounts, rownames(rlogcounts) %in% filtered_res[,1])
filtered_res_counts = t(scale(t(filtered_res_counts)))
Heatmap(filtered_res_counts[1:50,], name = "Z-Score", row_names_gp = gpar(fontsize = 8))

## <a name="workflow">Additional Workflows</a>

Now that you have ran some analysis on read count data, feel feel free to explore and revisit the previous workflows for creating readcounts, such as the standard or extended tutorials, or the snakemake tutorial.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://youtu.be/ChGfBR4do_Y>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)

## Conclusion

This Jupyter Notebook demonstrated a streamlined workflow for RNA-Seq differential gene expression (DGE) analysis using the DESeq2 package in R.  Starting with raw gene count data, the notebook guided users through key steps: library installation, data import and preprocessing (including handling of zero counts), experimental design specification, DESeq2 object creation, data normalization (using rlog transformation), PCA analysis for visualizing sample relationships, and contrast comparisons to identify differentially expressed genes between wildtype and bacteriophage-infected samples.  The results were then annotated using a feature table and exported for further analysis.  Finally, the notebook showcased visualizations such as volcano plots, MA plots, and heatmaps to explore the DGE results and identify potential candidate genes.  Links to supplementary tutorials expanding on data acquisition and processing were provided, allowing users to delve deeper into the RNA-Seq pipeline.  The notebook provides a comprehensive, practical introduction to RNA-Seq DGE analysis, suitable for both beginners and experienced users seeking a concise and efficient approach.

## Clean Up

Remember to move to the next notebook or shut down your instance if you are finished.